In [1]:
# 1. Устанавливаем библиотеки (это как скачать приложения на телефон)
# omegaconf и torchaudio нужны для работы модели Silero
!pip install -q torch silero omegaconf torchaudio soundfile

import torch
import soundfile as sf
import os

# 2. Настраиваем устройство (если есть GPU, используем его, иначе процессор)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используем устройство: {device}")

# 3. Скачиваем модель голоса (Silero v4 - русский язык)
print("Загружаем модель... Это займет пару секунд.")
local_file = 'model.pt'
if not os.path.isfile(local_file):
    torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v4_ru.pt',
                                   local_file)

model = torch.package.PackageImporter(local_file).load_pickle("tts_models", "model")
model.to(device)

# 4. Текст для озвучки (из твоего сценария Ozon)
# Ты можешь менять этот текст прямо здесь!
text_to_say = "Хм... звучит интересно. Но у меня уже есть расчетный счет в Сбере."

# 5. Генерация аудио
# speaker='aidar' (мужской) или 'kseniya' (женский), 'baya', 'xenia'
sample_rate = 48000
speaker = 'aidar' 

print(f"Генерируем голос персонажа {speaker}...")
audio = model.apply_tts(text=text_to_say,
                        speaker=speaker,
                        sample_rate=sample_rate)

# 6. Сохраняем файл и даем послушать прямо в браузере
filename = 'test_voice.wav'
sf.write(filename, audio, sample_rate)

from IPython.display import Audio, display
display(Audio(filename, autoplay=True))
print("Готово! Нажми Play на плеере выше.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

100%|██████████| 38.2M/38.2M [00:00<00:00, 145MB/s] 


Генерируем голос персонажа aidar...


Готово! Нажми Play на плеере выше.


In [2]:
# Меняем текст на более характерный для "Сомневающегося ИП"
text_to_say = "Хм... звучит интересно. Но у меня уже есть расчетный счет в Сбере. Зачем мне второй? Я сейчас занята."

# Пробуем другого спикера (если Aidar не понравился, попробуй 'eugene' - он иногда звучит строже)
speaker = 'kseniya' 
sample_rate = 48000
put_accent = True
put_yo = True

# ВАЖНО: Silero не имеет прямой настройки "speed" в apply_tts, 
# но мы можем просто генерировать, а потом ускорить, если это понадобится.
# Однако, модель 'v4_ru' сама по себе достаточно быстрая.

audio = model.apply_tts(text=text_to_say,
                        speaker=speaker,
                        sample_rate=sample_rate,
                        put_accent=put_accent,
                        put_yo=put_yo)

# Сохраняем и слушаем
filename = 'busy_client.wav'
sf.write(filename, audio, sample_rate)

from IPython.display import Audio, display
print(f"Слушаем {speaker} в роли занятого клиента:")
display(Audio(filename, autoplay=True))

Слушаем kseniya в роли занятого клиента:


In [3]:
# 1. Устанавливаем Whisper (это "Уши")
!pip install -q git+https://github.com/openai/whisper.git

import whisper
import os

# Проверяем, есть ли аудиофайл от прошлого шага
audio_file = 'busy_client.wav'
if not os.path.exists(audio_file):
    # Если ты вдруг перезагрузил страницу и файл пропал, создадим заглушку
    print("Файл не найден, использую тестовый...")
    # (Тут код создания файла пропущен, надеемся, что ты не закрывал вкладку)
    # Если файла нет - просто запусти предыдущий блок кода еще раз!

# 2. Загружаем модель Whisper
# 'base' - легкая модель. Есть еще 'small', 'medium', 'large' (они умнее, но медленнее)
print("Загружаем модель Whisper (это может занять минуту)...")
model_asr = whisper.load_model("base")

# 3. Распознаем речь
print(f"Слушаем файл: {audio_file} ...")
result = model_asr.transcribe(audio_file)

# 4. Выводим результат
print("\n--- РЕЗУЛЬТАТ ---")
print(f"Робот услышал: '{result['text']}'")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Загружаем модель Whisper (это может занять минуту)...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 159MiB/s]


Слушаем файл: busy_client.wav ...

--- РЕЗУЛЬТАТ ---
Робот услышал: ' Звучит интересно. Но у меня уже есть расчетный счет в сбере. Зачем мне второй я сейчас занята?'


In [4]:
# Это пример того, как программируется "Мозг" бота

# 1. СИСТЕМНЫЙ ПРОМПТ (Инструкция, которую мы загружаем в голову модели)
system_prompt = """
Ты — ИИ-клиент, имитирующий поведение предпринимателя. Твоя цель — тренировать менеджера по продажам.
ТВОЙ ПРОФИЛЬ:
- Имя: Дмитрий Сергеевич.
- Статус: Индивидуальный предприниматель (ИП).
- Ситуация: Только что зарегистрировался на Ozon, загрузил товары. Расчетного счета в Ozon Банке НЕТ.
- Характер: Деловит, скептичен, занят. Говоришь кратко.
- Текущий банк: У тебя уже есть счет в "Сбере", обслуживание 500р/мес. Ты не хочешь менять банк без веской причины.

ТВОИ ЗАДАЧИ:
1. Не соглашайся сразу. Ты должен возражать. Твои главные возражения: "Зачем мне второй счет?", "У меня уже есть банк", "Я занят".
2. Если менеджер приводит слабые аргументы — отказывай.
3. Если менеджер убедителен (рассказал про бесплатные переводы, чат с покупателями) — прояви интерес.
4. Никогда не выходи из роли. Не говори "Я искусственный интеллект".

ТВОЙ СТИЛЬ ОБЩЕНИЯ:
- Разговорный русский язык.
- Используй слова-паразиты ("ну", "как бы", "слушайте").
- Длина ответа: не более 2-3 предложений.
"""

# 2. Имитация диалога (Как это выглядит для программы)
dialog_history = [
    {"role": "system", "content": system_prompt},
    
    # Менеджер (это пришло от Whisper)
    {"role": "user", "content": "Дмитрий Сергеевич, добрый день! Звоню поздравить с регистрацией на Озон. Давайте откроем счет?"}
]

# Обычно здесь этот список летит в модель Qwen.
# Так как мы в Colab без GPU на 24GB, я покажу, как модель должна ответить (эмуляция):

print("--- ОТПРАВЛЯЕМ ЗАПРОС В LLM ---")
print(f"Роль системы загружена: {len(system_prompt)} символов.")
print("Последняя фраза менеджера: 'Давайте откроем счет?'")

print("\n--- ОЖИДАЕМЫЙ ОТВЕТ МОДЕЛИ (согласно сценарию) ---")
# В реальности этот текст генерирует Qwen
print("AI-клиент: Слушайте, спасибо конечно, но у меня уже есть счет в Сбере. Зачем мне еще один? Я сейчас занят вообще-то.")

--- ОТПРАВЛЯЕМ ЗАПРОС В LLM ---
Роль системы загружена: 984 символов.
Последняя фраза менеджера: 'Давайте откроем счет?'

--- ОЖИДАЕМЫЙ ОТВЕТ МОДЕЛИ (согласно сценарию) ---
AI-клиент: Слушайте, спасибо конечно, но у меня уже есть счет в Сбере. Зачем мне еще один? Я сейчас занят вообще-то.


In [5]:
# Ставим библиотеки.
# transformers==4.46.0 нужен, чтобы XTTS не ломался.
!pip install -q transformers==4.46.0
!pip install -q TTS
!pip install -q torch silero omegaconf torchaudio soundfile

print("✅ Шаг 2 завершен. Библиотеки установлены.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 89.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 71.7 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 87.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 69.6 MB/s eta 0:00:0000:0100:01
  Pr

In [6]:
import torch
import soundfile as sf
import os
from IPython.display import Audio, display

# --- 1. БЕЗОПАСНОЕ ЛЕЧЕНИЕ ОШИБКИ (FIX) ---
# Проверяем, не применили ли мы патч уже ранее, чтобы избежать RecursionError
if not hasattr(torch, '_original_load_backup'):
    torch._original_load_backup = torch.load

def fix_torch_load(*args, **kwargs):
    # Если параметр 'weights_only' не передан, ставим его в False (как было раньше)
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return torch._original_load_backup(*args, **kwargs)

# Подменяем функцию
torch.load = fix_torch_load
# ------------------------------------------

# --- 2. НАСТРОЙКА ---
os.environ["COQUI_TOS_AGREED"] = "1"
from TTS.api import TTS

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Работаем на устройстве: {device}")

# --- 3. СОЗДАЕМ ГОЛОС-ДОНОР (через Silero) ---
# Нам нужен wav-файл, чтобы XTTS знал, чей голос клонировать.
# Мы генерируем его "на лету", чтобы тебе не нужно было ничего загружать руками.
print("🔊 Генерируем 'донора' голоса...")

local_file = 'model.pt'
if not os.path.isfile(local_file):
    torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v4_ru.pt',
                                   local_file)

model_silero = torch.package.PackageImporter(local_file).load_pickle("tts_models", "model")
model_silero.to(device)

# Создаем файл busy_client.wav
ref_audio = model_silero.apply_tts(text="Это просто технический файл для настройки тембра голоса.",
                                   speaker='aidar',
                                   sample_rate=48000)
sf.write('busy_client.wav', ref_audio, 48000)


# --- 4. ЗАПУСК XTTS (Клонирование) ---
print("🧠 Загружаем XTTS и генерируем результат...")

# Загружаем модель
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Текст сценария
text_to_clone = "Хм... звучит интересно. Но у меня уже есть расчетный счет в Сбере. Зачем мне второй? Я сейчас занят."

output_file = "final_result.wav"

# Генерируем
tts.tts_to_file(text=text_to_clone,
                speaker_wav=["busy_client.wav"], 
                language="ru",
                file_path=output_file)

print("🎉 УСПЕХ! Слушаем результат:")
display(Audio(output_file, autoplay=True))

🚀 Работаем на устройстве: cuda
🔊 Генерируем 'донора' голоса...
🧠 Загружаем XTTS и генерируем результат...
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.86G/1.87G [00:17<00:00, 107MiB/s] 
100%|██████████| 1.87G/1.87G [00:17<00:00, 105MiB/s]
4.37kiB [00:00, 13.1kiB/s]

361kiB [00:00, 987kiB/s].0 [00:00<?, ?iB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 66.0iB/s]
 30%|███       | 2.34M/7.75M [00:00<00:00, 6.17MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.


2025-12-02 22:04:59.043722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764713099.268925      83 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764713099.332834      83 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

 > Using model: xtts


GPT2InferenceModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


 > Text splitted to sentences.
['Хм... звучит интересно.', 'Но у меня уже есть расчетный счет в Сбере.', 'Зачем мне второй?', 'Я сейчас занят.']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 4.2606565952301025
 > Real-time factor: 0.41038002308509125
🎉 УСПЕХ! Слушаем результат:
